# SymPy on Python
* http://docs.sympy.org/latest/modules/physics/mechanics/masses.html

## Rigid Body

In [3]:
from sympy import Symbol
from sympy.physics.mechanics import ReferenceFrame, Point, RigidBody
from sympy.physics.mechanics import outer

m = Symbol('m')
A = ReferenceFrame('A')
P = Point('P')
I = outer(A.x, A.x)

# Create a rigid body
B = RigidBody('B', P, A, m, (I, P))


## Lagrangian

In [18]:
from sympy import symbols
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame, outer
from sympy.physics.mechanics import RigidBody, Particle, mechanics_printing
from sympy.physics.mechanics import kinetic_energy, potential_energy, Lagrangian, Point

mechanics_printing()
m, M, l1, g, h, H = symbols('m M l1 g h H')
omega = dynamicsymbols('omega')
N = ReferenceFrame('N')
O = Point('O')
O.set_vel(N, 0 * N.x)
Ac = O.locatenew('Ac', l1 * N.x)
P = Ac.locatenew('P', l1 * N.x)
a = ReferenceFrame('a')
a.set_ang_vel(N, omega * N.z)
Ac.v2pt_theory(O, N, a)
P.v2pt_theory(O, N, a)
Pa = Particle('Pa', P, m)
I = outer(N.z, N.z)
A = RigidBody('A', Ac, a, M, (I, Ac))

In [19]:
kinetic_energy(N, Pa)

In [20]:
kinetic_energy(N, Pa, A)

In [21]:
Pa.potential_energy = m * g * h
A.potential_energy = M * g * H
potential_energy(A, Pa)

In [28]:
Lagrangian(N, Pa, A)

## Lagrange’s Method in Physics/Mechanics
* http://docs.sympy.org/latest/modules/physics/mechanics/lagrange.html

In [31]:
from sympy.physics.mechanics import *

q1, q2 = dynamicsymbols('q1 q2')
q1d, q2d = dynamicsymbols('q1 q2', 1)
L = q1d**2 + q2d**2

In [34]:
LM = LagrangesMethod(L, [q1, q2])

In [39]:
mechanics_printing(pretty_print=True)
LM.form_lagranges_equations()

⎡2⋅q₁̈⎤
⎢     ⎥
⎣2⋅q₂̈⎦

In [40]:
LM.mass_matrix

⎡2  0⎤
⎢    ⎥
⎣0  2⎦

In [41]:
LM.forcing

⎡0⎤
⎢ ⎥
⎣0⎦

In [42]:
LM = LagrangesMethod(L, [q1, q2], hol_coneqs=[q1 - q2])

In [43]:
LM.form_lagranges_equations()

⎡lam₁ + 2⋅q₁̈ ⎤
⎢             ⎥
⎣-lam₁ + 2⋅q₂̈⎦

In [44]:
LM.mass_matrix_full

⎡1  0  0  0   0 ⎤
⎢               ⎥
⎢0  1  0  0   0 ⎥
⎢               ⎥
⎢0  0  2  0   -1⎥
⎢               ⎥
⎢0  0  0  2   1 ⎥
⎢               ⎥
⎣0  0  1  -1  0 ⎦

In [45]:
LM.forcing_full

⎡q₁̇⎤
⎢   ⎥
⎢q₂̇⎥
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎣ 0 ⎦

In [46]:
N = ReferenceFrame('N')
P = Point('P')
P.set_vel(N, q1d * N.x)
FL = [(P, 7 * N.x)]
LM = LagrangesMethod(L, [q1, q2], forcelist=FL, frame=N)
LM.form_lagranges_equations()


⎡2⋅q₁̈ - 7⎤
⎢         ⎥
⎣  2⋅q₂̈  ⎦

## Examples for Physics/Mechanics
* http://docs.sympy.org/latest/modules/physics/mechanics/examples.html

#### Nonminimal Coordinates Pendulum (Lagrange's Method)

![](http://docs.sympy.org/latest/_images/pendulum_nonmin.svg)

In [47]:
from sympy.physics.mechanics import *
from sympy import symbols, atan, Matrix

q1, q2 = dynamicsymbols('q1:3')
q1d, q2d = dynamicsymbols('q1:3', level=1)
L, m, g, t = symbols('L, m, g, t')

In [48]:
 # Compose World Frame
N = ReferenceFrame('N')
pN = Point('N*')
pN.set_vel(N, 0)
# A.x is along the pendulum
theta1 = atan(q2/q1)
A = N.orientnew('A', 'axis', [theta1, N.z])

In [49]:
# Create point P, the pendulum mass
P = pN.locatenew('P1', q1*N.x + q2*N.y)
P.set_vel(N, P.pos_from(pN).dt(N))
pP = Particle('pP', P, m)

In [50]:
# Holonomic Constraint Equations
f_c = Matrix([q1**2 + q2**2 - L**2])


In [51]:
# Input the force resultant at P
R = m*g*N.x

In [53]:
# Calculate the lagrangian, and form the equations of motion
Lag = Lagrangian(N, pP)
LM = LagrangesMethod(Lag, [q1, q2], hol_coneqs=f_c, forcelist=[(P, R)], frame=N)
lag_eqs = LM.form_lagranges_equations()

In [54]:
# Compose operating point
op_point = {q1: L, q2: 0, q1d: 0, q2d: 0, q1d.diff(t): 0, q2d.diff(t): 0}

In [55]:
# Solve for multiplier operating point
lam_op = LM.solve_multipliers(op_point=op_point)

In [56]:
op_point.update(lam_op)
# Perform the Linearization
A, B, inp_vec = LM.linearize([q2], [q2d], [q1], [q1d], op_point=op_point, A_and_B=True)
A


⎡ 0   1⎤
⎢      ⎥
⎢-g    ⎥
⎢───  0⎥
⎣ L    ⎦

In [57]:
B

[]

__Result : __

$$
\begin{bmatrix}
  \dot{q_2} \\ \ddot{q_2}
\end{bmatrix}
=
\begin{bmatrix}
  0 & 1 \\
  \frac{-g}{L} & 0
\end{bmatrix}
\begin{bmatrix}
  q_2 \\ \dot{q_2}
\end{bmatrix}
$$

In [59]:
%whos

Variable                      Type                  Data/Info
-------------------------------------------------------------
A                             MutableDenseMatrix    Matrix([[0, 1], [-g/L, 0]])
Ac                            Point                 Ac
B                             MutableDenseMatrix    Matrix(0, 0, [])
Body                          type                  <class 'sympy.physics.mechanics.body.Body'>
CoordinateSym                 ManagedProperties     <class 'sympy.physics.vec<...>tor.frame.CoordinateSym'>
Dyadic                        type                  <class 'sympy.physics.vector.dyadic.Dyadic'>
FL                            list                  n=1
H                             Symbol                H
I                             Dyadic                (N.z|N.z)
KanesMethod                   type                  <class 'sympy.physics.mec<...>hanics.kane.KanesMethod'>
L                             Symbol                L
LM                            Lagrang

In [60]:
%history

from sympy import Symbol
from sympy.physics.mechanics import ReferenceFrame, Point, RigidBody
from sympy.physics.mechanics import outer

m = Symbol('m')
A = ReferenceFrame('A')
P = Point('P')
I = outer(A.x, A.x)

# Create a rigid body
B = RigidBody('B', P, A, m, (I, P))
from sympy import Symbol
from sympy.physics.mechanics import ReferenceFrame, Point, RigidBody
from sympy.physics.mechanics import outer

m = Symbol('m')
A = ReferenceFrame('A')
P = Point('P')
I = outer(A.x, A.x)

# Create a rigid body
B = RigidBody('B', P, A, m, (I, P))
B
from sympy import Symbol
from sympy.physics.mechanics import ReferenceFrame, Point, RigidBody
from sympy.physics.mechanics import outer

m = Symbol('m')
A = ReferenceFrame('A')
P = Point('P')
I = outer(A.x, A.x)

# Create a rigid body
B = RigidBody('B', P, A, m, (I, P))
kinetic_energy(A, P)
from symp.physics.mechanics import kinetic_energy, potential_energy

kinetic_energy(A, P)
from sympy.physics.mechanics import kinetic_energy, potential_energy

kine